# The Evolution of Java: 9, 10, 11, and 12
This notebook contains my notes from the *Evolution of Java: 9, 10, 11, and 12* talk by Venkat Subramaniam at UberConf 2019. In this talk, he discussed and demoed the various new language and API features in Java 9, 10, and 11.

### Release Cycle
Java is now on a 6-month release cycle. This will change how a lot of organizations need to do upgrades. Instead of an every-5-years upgrade, the LTS releases will have shorter support periods.

In this release cycle, features are still going to take the time they will take. Type inference in Java took 5 years until it was finally released in Java 10. New language features aren't going to magically get faster, but this faster release cycle will allow things like new API improvements to be added without having to wait for years.

### Private Methods in Interfaces (9)
Java 8 added default methods in interfaces:

In [7]:
public interface MyInterface {
    default int getNumber() {
        return 42;
    }
}

public class MyClass implements MyInterface {}

MyClass myClass = new MyClass();
myClass.getNumber();

42

Java 9 takes this further and adds the ability to have private methods in interfaces:

In [8]:
public interface PrivateMethodsInterface {
    default int getNumber() {
        return getTheNumber();
    }
    
    private int getTheNumber() {
        return 42;
    }
}

public class PrivateMethodsClass implements PrivateMethodsInterface {}

PrivateMethodsClass privateMethodsClass = new PrivateMethodsClass();
privateMethodsClass.getNumber();

42

### Try-with-Resources Improvements (9)
Java has had try-with-resources since Java 7:

In [9]:
public class Resource implements AutoCloseable {
    public void op1() {
        System.out.println("Hello");
    }
    public void close() {
        System.out.println("Closing");
    }
}

try(Resource resource = new Resource();) {
    resource.op1();
}

Hello
Closing


You can now use an already existing resource, provided it is *final* or *effectively final*

In [13]:
public void doSomething() {
    final Resource resource2 = new Resource();
    try(resource2) {
        resource.op1();
    }    
}
doSomething();

Hello
Closing


### *_* is a Keyword (9)
Other languages often have the variable name *_* mean "I don't care about this value". Java previously let you use that as a variable. Java 9 made *_* a keyword and won't let you use it as an identifier anymore. 

This is in preparation for adding support for adding language support for the "I don't care about this value" identifier in the future. You probably shouldn't have variables named *_* anyway...

In [15]:
String _ = "Hello"

CompilationException: 

### List.of, Map.of, Set.of (9)
List.of is a nice way to create a list from known elements. It is better than Arrays.asList because it creates truly immutable lists. Here is an example of how Arrays.asList is not immutable:

In [9]:
import java.util.List;
import java.util.Arrays;

List<Integer> numbers = Arrays.asList(1,2,3,4);
System.out.println(numbers);
numbers.add(0);

[1, 2, 3, 4]


EvalException: null

The above code behaves as expected, throwing an *UnsupportedOperationException*. The following doesn't behave as expected, however:

In [10]:
numbers.set(0, 5);
System.out.println(numbers);

[5, 2, 3, 4]


It allows you to change elements in the list, so the collection is not immutable. Java 9 adds new methods that return collections that are actually immutable:

In [14]:
numbers = List.of(1,2,3,4);
System.out.println(numbers);
numbers.set(0, 5); // This should fail

[1, 2, 3, 4]


EvalException: null

There are also corresponding Set.of and Map.of methods:

In [20]:
Set<Integer> someSet = Set.of(1,2,3,4);
System.out.println(someSet);

// This varargs function expects alternating key/value pairs
Map<String, Integer> someMap = Map.of(
    "Zero", 0,
    "One", 1,
    "Two", 2
);
System.out.println(someMap);

[1, 2, 3, 4]
{Zero=0, One=1, Two=2}


### Streams Improvements (9 and 10)
The *takeWhile* function is a nice addition to the streams API. Venkat said it can be thought of as the "break" in a traditional for loop. You can perform an action while a condition is true, then break out of the iteration:

In [24]:
import java.util.stream.*;

List.of(1,2,3,4,5,6,7)
    .stream()
    .takeWhile(i -> i < 5)
    .collect(Collectors.toList());

[1, 2, 3, 4]

There is also a corresponding *dropWhile* function:

In [25]:
List.of(1,2,3,4,5,6,7)
    .stream()
    .dropWhile(i -> i < 5)
    .collect(Collectors.toList());

[5, 6, 7]

IntStream also has several new additions. In adddition to the existing open *range* function, there is now a closed *rangeClosed* function:

In [33]:
// Regular open range
List<Integer> numbers = IntStream.range(0,5)
    .boxed()
    .collect(Collectors.toList());
System.out.println(numbers);
    
// New closed range
numbers = IntStream.rangeClosed(0,5)
    .boxed()
    .collect(Collectors.toList());
System.out.println(numbers);

[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4, 5]


There is also a new *iterate* method that allows you to do fancy looping variable steps like you could do in a for loop. Just like the for loop, it takes three arguments: An initial seed, a predicate, and a function to update the value after each predicate evaluation:

In [36]:
IntStream.iterate(0, i -> i < 20, i -> i + 3)
    .boxed()
    .collect(Collectors.toList());

[0, 3, 6, 9, 12, 15, 18]

The *Collectors.toList()* function returns a mutable list. You can make an immutable list from a Stream with the new *Collectors.toUnmodifiableList()*. There are corresponding functions *Collectors.toUnmodifiableSet()* and Collectors.toUnmodifiableMap()*

In [50]:
List.of(1,2,3,4)
    .stream()
    .collect(Collectors.toUnmodifiableList());

[1, 2, 3, 4]

### Optional Improvements (9, 10, and 11)
Optional was introduced in Java 8. The speaker said that while this is great to use as the return value in methods, it should probably not be used as a parameter to a method. He said that method overloading is a much better way to make a method deal with an optional parameter. If Java had default method parameters you wouldn't even have to do that... :)

There are some new methods in Optional. The *orElse* method returns the value if present or a default that you specify:

In [39]:
Optional<String> myOptional = Optional.empty();
myOptional.orElse("This is the default")

This is the default

There is a corresponding method you can use to throw an exception:

In [43]:
myOptional.orElseThrow(() -> new RuntimeException("Optional was missing"));

EvalException: Optional was missing

An Optional also kind of can be thought of a stream that contains 0 or 1 values, so now the API lets you turn on into a Stream:

In [51]:
myOptional = Optional.of("Hello");
myOptional.stream()
    .collect(Collectors.toUnmodifiableList());

[Hello]

Also, as a complement to the already-existing Optional.isPresent(), you now have Optional.isEmpty():

In [53]:
myOptional = Optional.empty();
if(myOptional.isEmpty()) {
    System.out.println("Optional is missing");
}

Optional is missing


### JShell REPL (9)
Java 9 introduces a great new tool called JShell, which is a REPL for Java. It is awesome and works great for executing tidbits of code. As a matter of fact, JShell is what powers the Java kernel that this Jupyter notebook is using right now!

### Type Inference (10)
Type inference lets you leave off the initial type signature when it can be inferred by the compiler. This only works for local variables.

In [49]:
var list = List.of(1,2,3,4);
System.out.println(list);

[1, 2, 3, 4]


### Easy Read and Write String (11)
The Java NIO API has some easy static functions added for reading and writing files:

In [66]:
import java.nio.file.Path;
import java.nio.file.Files;

// Write string
var tempFilePath = Path.of(System.getProperty("java.io.tmpdir"), "testfile");
Files.writeString(tempFilePath, "Hello!");

// Read string
var fileContents = Files.readString(tempFilePath);
System.out.println(fileContents);

// Cleanup the temp file
Files.delete(tempFilePath);

Hello!
